In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
hr=pd.read_csv('/kaggle/input/hr-analytics/HR_comma_sep.csv')

In [ ]:
hr.head()

In [ ]:
hr.info()

In [ ]:
hr.left.value_counts()/len(hr.left)

In [ ]:
hr.columns

In [ ]:
hr.Department.value_counts()

In [ ]:
hr.salary.value_counts()

In [ ]:
import seaborn as sb
from matplotlib import pyplot as pt
pd.set_option('display.max_columns',None)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
sb.countplot(hr.salary,hue=hr['left'])

In [ ]:
sb.countplot(hr.Department,hue=hr['left'])
pt.xticks(rotation=90)


In [ ]:
hr.corr()['left'].sort_values(ascending=False)

In [ ]:
hr.Work_accident.value_counts()

In [ ]:
sb.distplot(hr.satisfaction_level)

In [ ]:
sb.distplot(1/(hr.satisfaction_level))

In [ ]:
sb.distplot(np.sqrt(hr.time_spend_company),kde=False)

In [ ]:
#time_spend,satisfaction_level,work_accident,salary,department

In [ ]:
salry=pd.get_dummies(hr.salary,drop_first=True)
dept=pd.get_dummies(hr.Department,drop_first=True)

In [ ]:
hrd=hr.copy()

In [ ]:
hrd=pd.concat([hrd,salry,dept],axis=1)

In [ ]:
hrd.head()

In [ ]:
hrd.drop(['salary','Department'],axis=1,inplace=True)

In [ ]:
hrd.head()

In [ ]:
X=hrd.[hrd['satisfaction_level','time_spend_company','Work_accident',
     'low','medium','RandD','accounting','hr','management','marketing','product_mng','sales','support','technical']]

In [ ]:
x=hrd.drop(['last_evaluation','number_project','average_montly_hours','left','promotion_last_5years'],axis=1)

In [ ]:
x.head()

In [ ]:
y=hrd['left'].values

In [ ]:
x=x.values

In [ ]:
xtrn,xtst,ytrn,ytst=train_test_split(x,y,test_size=0.25,random_state=0)

In [ ]:
xtrn.shape,ytrn.shape,xtst.shape,ytst.shape

In [ ]:
st=StandardScaler()
xtrnstd=st.fit_transform(xtrn)
xtststd=st.transform(xtst)

In [ ]:
logstc=LogisticRegression(random_state=0)

In [ ]:
logstc.fit(xtrnstd,ytrn)

In [ ]:
ylogprd=logstc.predict(xtststd)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(ytst,ylogprd))

In [ ]:
tre=RandomForestClassifier(n_estimators=150,random_state=0)
tre.fit(xtrnstd,ytrn)
ytreprd=tre.predict(xtststd)

In [ ]:
print(classification_report(ytst,ytreprd))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(ytst,ytreprd)

In [ ]:
xg=XGBClassifier(n_estimators=150,random_state=0)
xg.fit(xtrnstd,ytrn)
xgprd=xg.predict(xtststd)

In [ ]:
print(classification_report(ytst,xgprd))

In [ ]:
confusion_matrix(ytst,xgprd)

In [ ]:
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler

In [ ]:
smple=RandomOverSampler(sampling_strategy=1,random_state=0)

In [ ]:
xsmple,ysmple=smple.fit_sample(xtrnstd,ytrn)

In [ ]:
sample0=0
sample1=0
for i in range(0,len(ysmple)):
    if ysmple[i]==0:
        sample0+=1
    else:
        sample1+=1
    

In [ ]:
sample0,sample1

In [ ]:
sm=SMOTETomek(random_state=0)

In [ ]:
xtrnbalnced,ytrnbalnced=sm.fit_sample(xtrnstd,ytrn)

In [ ]:
ytrnbalnced.shape

In [ ]:
ytrn.shape

In [ ]:
cnt=0
cnt1=0
for i in range(0,len(ytrn)):
    if ytrn[i]==0:
        cnt+=1
    else:
        cnt1+=1
    

In [ ]:
cnt,cnt1

In [ ]:
banlced0=0
banlced1=0
for i in range(0,len(ytrnbalnced)):
    if ytrnbalnced[i]==0:
        banlced0+=1
    else:
        banlced1+=1

In [ ]:
banlced0,banlced1


In [ ]:
regrss=LogisticRegression(random_state=0)

In [ ]:
regrss.fit(xsmple,ysmple)

In [ ]:
ysampleprd=regrss.predict(xtststd)

In [ ]:
print(classification_report(ytst,ysampleprd))

In [ ]:
frst=RandomForestClassifier(n_estimators=150,random_state=0)
frst.fit(xsmple,ysmple)
yfrstprd=frst.predict(xtststd)

In [ ]:
print(classification_report(ytst,yfrstprd))

In [ ]:
boost=XGBClassifier(n_estimators=150,random_state=0)
boost.fit(xsmple,ysmple)
boostprd=boost.predict(xtststd)

In [ ]:
print(classification_report(ytst,boostprd))

In [ ]:
confusion_matrix(ytst,boostprd),confusion_matrix(ytst,xgprd)

In [ ]:
xgsmot=XGBClassifier(n_estimators=150)
xgsmot.fit(xtrnbalnced,ytrnbalnced)
xgsmotprd=xgsmot.predict(xtststd)

In [ ]:
print(classification_report(ytst,xgsmotprd))

In [ ]:
ramnd=RandomForestClassifier(n_estimators=150)
ramnd.fit(xtrnbalnced,ytrnbalnced)
rmndprd=ramnd.predict(xtststd)

In [ ]:
print(classification_report(ytst,rmndprd))

In [ ]:
confusion_matrix(ytst,rmndprd),confusion_matrix(ytst,yfrstprd)

In [ ]:
hrd.head()

In [ ]:
featrs=hrd.drop(['left'],axis=1)

In [ ]:
featrstr,featrstst,yftrn,yftst=train_test_split(featrs,y,test_size=0.25,random_state=0)

In [ ]:
fsttre=RandomForestClassifier(n_estimators=150)
fsttre.fit(featrstr,yftrn)
fsttreprd=fsttre.predict(featrstst)

In [ ]:
print(classification_report(yftst,fsttreprd))

In [ ]:
confusion_matrix(yftst,fsttreprd)

In [ ]:
xgclsf=XGBClassifier(n_estimators=150,random_state=0)
xgclsf.fit(featrstr,yftrn)
xgclsfprd=xgclsf.predict(featrstst)

In [ ]:
print(classification_report(yftst,xgclsfprd))

In [ ]:
confusion_matrix(yftst,xgclsfprd)